<a href="https://www.kaggle.com/code/seyed0123/chess?scriptVersionId=286979568" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# prossesing the data

In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["TOKENIZERS_PARALLELISM"] = "false"


def load_all_txt(input_dir):
    all_text = ""

    for filename in os.listdir(input_dir):
        if filename.endswith(".txt"):
            file_path = os.path.join(input_dir, filename)
            with open(file_path, "r", encoding="utf-8") as f:
                all_text += f.read() + "\n"

    return all_text

In [2]:
input_directory = r"/kaggle/input/chess-analyse-books"
full_text = load_all_txt(input_directory)
print(len(full_text))

5556076


# load and test model

In [3]:
!pip install transformers accelerate pyarrow==14.0.2 datasets==2.18.0 python-chess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 70.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.8 MB/s eta 0:00:0

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, StoppingCriteria, StoppingCriteriaList
import torch

model_name = "Qwen/Qwen3-1.7B"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
    padding_side="left"
)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype=torch.float32,
    device_map="auto"
)
model.eval()


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/622M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=2048, out_features=6144, bias=False)
          (up_proj): Linear(in_features=2048, out_features=6144, bias=False)
          (down_proj): Linear(in_features=6144, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): Qwe

In [5]:
class StopAtSentence(StoppingCriteria):
    def __call__(self, input_ids, scores, **kwargs):
        # Check last token
        last_token = input_ids[0][-1].item()
        # Stop if EOS or common sentence-ending token (".", "?", "!")
        if last_token == tokenizer.eos_token_id:
            return True
        decoded = tokenizer.decode([last_token], skip_special_tokens=True)
        if decoded.strip() in {".", "?", "!"}:
            return True
        return False

def chat_qwen3_direct(prompt: str, max_new_tokens: int = 256) -> str:
    messages = [{"role": "user", "content": prompt}]
    
    # Apply chat template WITHOUT thinking
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False  # 🔥 Disable thinking for factual answers
    )
    
    inputs = tokenizer([text], return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.5,      # Lower = more focused
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            # stopping_criteria=StoppingCriteriaList([StopAtSentence()])  # Optional
        )
    
    # Extract only the new text
    input_len = inputs.input_ids.shape[1]
    response = tokenizer.decode(outputs[0][input_len:], skip_special_tokens=True)
    
    # Clean: remove any trailing incomplete sentence
    for end in [".", "?", "!"]:
        if end in response:
            response = response[:response.rfind(end)+1]
            break
    else:
        # No sentence end? Return first line or truncate
        response = response.split("\n")[0].strip()
    
    return response.strip()

## Test the model output on simple question

In [6]:
prompt = "Explain why controlling the center is important in chess in 2–3 sentences."
answer = chat_qwen3_direct(prompt)
print("💬 Answer:", answer)

💬 Answer: Controlling the center in chess is crucial because it gives a player a significant advantage in terms of piece mobility, attack, and development. The center is where the most active pieces are, and controlling it allows for better coordination and faster progress on the board. A strong center control often leads to a more dynamic and flexible game, giving the player a greater chance of winning.


# fine tune the model

In [7]:
def chunk_text(text, tokenizer, max_length):
    # Use batched tokenization to avoid huge tensors
    tokens = tokenizer(
        text,
        return_tensors="pt",
        add_special_tokens=False,  # Avoid extra BOS/EOS in middle of book
        truncation=False,
        verbose=False
    )["input_ids"][0]

    chunks = []
    # Use full chunks only (avoid tiny trailing fragments)
    for i in range(0, len(tokens) - max_length + 1, max_length):
        chunk = tokens[i:i + max_length].tolist()
        chunks.append({"input_ids": chunk})
    
    # Optional: add last partial chunk if long enough
    remainder = len(tokens) % max_length
    if remainder > 64:  # only if >64 tokens
        chunks.append({"input_ids": tokens[-remainder:].tolist()})
    
    return chunks

## prepare the data for training

In [8]:
from peft import LoraConfig, get_peft_model, TaskType,PeftModel
lora_config = LoraConfig(
    r=32,                # LoRA rank
    lora_alpha=32,       # LoRA alpha
    target_modules=["q_proj", "v_proj"],  # common modules for LLaMA/Qwen
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# Wrap model with LoRA
model = get_peft_model(model, lora_config)

In [9]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset

max_length = 256

data = chunk_text(full_text, tokenizer, max_length)
dataset = Dataset.from_list(data)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,                              
    pad_to_multiple_of=8                    
)

In [10]:
training_args = TrainingArguments(
    output_dir="./chess-qwen",
    run_name="chess-qwen-run-1",

    
    per_device_train_batch_size=2,    
    gradient_accumulation_steps=16,   

   
    num_train_epochs=6,
    learning_rate=2e-4,                
    fp16=True,                      
    bf16=False,                        

    
    save_steps=2000,                   
    logging_steps=200,                  
    save_total_limit=1,                 

    # 🚀 Other speedups
    optim="adamw_torch",                
    report_to="none",                  
    dataloader_num_workers=4,           
    remove_unused_columns=True,
)


# 5. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


## training loop

In [11]:
print('start training')
trainer.train()

print("Merging LoRA adapters into base model...")
model = model.merge_and_unload()
model.save_pretrained(
    "./chess-qwen-adapted",
    safe_serialization=True
)
tokenizer.save_pretrained("./chess-qwen-adapted")

start training


Step,Training Loss
200,2.869700
400,2.628200
600,2.540000
800,2.486700
1000,2.453600
1200,2.425400
1400,2.410100


Merging LoRA adapters into base model...


('./chess-qwen-adapted/tokenizer_config.json',
 './chess-qwen-adapted/special_tokens_map.json',
 './chess-qwen-adapted/chat_template.jinja',
 './chess-qwen-adapted/vocab.json',
 './chess-qwen-adapted/merges.txt',
 './chess-qwen-adapted/added_tokens.json',
 './chess-qwen-adapted/tokenizer.json')

In [12]:
del trainer
del model
del data_collator
del dataset
del tokenizer

torch.cuda.empty_cache()

# test the adapted model on same simple question

In [13]:
model_name = "/kaggle/working/chess-qwen-adapted"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
    padding_side="left"
)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype=torch.float16,
    device_map="auto"
)
model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=2048, out_features=6144, bias=False)
          (up_proj): Linear(in_features=2048, out_features=6144, bias=False)
          (down_proj): Linear(in_features=6144, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): Qwe

In [14]:
prompt = "Explain why controlling the center is important in chess in 2–3 sentences."
answer = chat_qwen3_direct(prompt)
print("💬 Answer:", answer)

💬 Answer: Controlling the center is crucial in chess because it provides a strong positional foundation, allows for better piece coordination, and creates more dynamic play. It also enables a player to exert influence over the board and restrict the opponent's development, making it easier to create an advantage or force a draw.


# Quantizing the model using llama-cpp

In [15]:
!pip install sentencepiece
!pip install "llama-cpp-python>=0.2.0"
!git clone https://github.com/ggerganov/llama.cpp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 40.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.0 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.16-cp311-cp311-linux_x86_64.whl size=4503249 sha256=b2b4430bc7329a71f03140c7d390b36967dbc5684cd467794f0f3b7d49d137dd
  Stored in directory: /root/.cache/pip/wheels/d8/5b/e5/a7d4b5765da347d314e8155197440c9995a962f8e4a5f52b23
Successfully built llama-cpp-python
Cloning into 'llama.cpp'...
remote: Enumerating objects: 72632, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 72632 (delta 44), reused 32 (delta 32), pack-reused 72547 (from 2)
Receiving objects: 100% (72632/72632), 246.46 MiB | 40.50 MiB/s, done.
Resolving deltas: 

In [16]:
!python llama.cpp/convert_hf_to_gguf.py \
    /kaggle/working/chess-qwen-adapted \
    --outfile /kaggle/working/chess-fp16.gguf \
    --outtype f16

Writing:   0%|                                   | 0.00/3.44G [00:00<?, ?byte/s]/kaggle/working/llama.cpp/convert_hf_to_gguf.py:10431: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:203.)
  return torch.from_numpy(byteswap_tensor(tensor.mmap_bytes(), numpy_dtype)).view(dtype).reshape(tensor.shape)
Writing: 100%|███████████████████████████| 3.44G/3.44G [00:11<00:00, 309Mbyte/s]


In [17]:
!cd llama.cpp && mkdir -p build && cd build && cmake .. && make -j

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
CMAKE_BUILD_TYPE=Release
-- Found Git: /usr/bin/git (found version "2.34.1")
-- The ASM compiler identification is GNU
-- Found assembler: /usr/bin/cc
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE
-- Warning: ccache not found - consider installing it for faster compilation or disable this warning with GGML_CCACHE=OFF
-- CMAKE_SYSTEM_PROCESSOR: x86_64
-- GGML_SYSTEM_ARCH: x86
-- Including CPU backend
-- Found OpenMP_C: 

In [18]:
!llama.cpp/build/bin/llama-quantize \
    /kaggle/working/chess-fp16.gguf \
    /kaggle/working/chess-q8_0.gguf \
    q8_0

main: build = 7469 (9ce64aed7)
main: built with GNU 11.4.0 for Linux x86_64
main: quantizing '/kaggle/working/chess-fp16.gguf' to '/kaggle/working/chess-q8_0.gguf' as Q8_0
llama_model_loader: loaded meta data with 29 key-value pairs and 310 tensors from /kaggle/working/chess-fp16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen3
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                     general.sampling.top_k i32              = 20
llama_model_loader: - kv   3:                     general.sampling.top_p f32              = 0.950000
llama_model_loader: - kv   4:                      general.sampling.temp f32              = 0.600000
llama_model_loader: - kv   5:                               general.name str              = Ches

# compare quantize model and two pipelines

## define the funcs

In [19]:
import chess
import chess.svg
import ipywidgets as widgets
from IPython.display import display

def board_widget(fen, size=350):
    board = chess.Board(fen)
    svg = chess.svg.board(board, size=size)
    return widgets.HTML(value=svg)


def labeled_board(title, fen):
    label = widgets.HTML(
        value=f"""
        <div style="
            text-align:center;
            font-weight:bold;
            margin-bottom:6px;
            font-size:16px;
        ">
            {title}
        </div>
        """
    )

    board = board_widget(fen)

    return widgets.VBox(
        [label, board],
        layout=widgets.Layout(align_items="center")
    )


def show_analysis(sample, llm_output):
    before_block = labeled_board("Before position", sample["before"]["fen"])
    after_block  = labeled_board("After position", sample["after"]["fen"])

    boards_row = widgets.HBox(
        [before_block, after_block],
        layout=widgets.Layout(
            justify_content="center",
            gap="40px"
        )
    )

    # Move type badge (color-coded)
    move_type = sample.get("move_type", "unknown").capitalize()
    move_colors = {
        "Blunder": "#d9534f",
        "Mistake": "#f0ad4e",
        "Inaccuracy": "#f7c948",
        "Normal": "#5bc0de",
        "Good": "#5cb85c"
    }
    color = move_colors.get(move_type, "#999")

    move_type_badge = widgets.HTML(
        value=f"""
        <div style="
            display:inline-block;
            margin:12px 0 6px 0;
            padding:4px 10px;
            border-radius:12px;
            font-size:13px;
            font-weight:bold;
            color:white;
            background:{color};
        ">
            {move_type}
        </div>
        """
    )

    # LLM text (left-aligned, symmetric)
    text = widgets.HTML(
        value=f"""
        <div style="
            margin-top:6px;
            padding:8px 12px;
            border:1px solid #ddd;
            border-radius:6px;
            font-family:Arial, sans-serif;
            font-size:14px;
            line-height:1.5;
            text-align:left;
            background:#fafafa;
        ">
            <div style="margin-left:16px;">
                {llm_output}
            </div>
        </div>
        """
    )

    display(
        widgets.VBox(
            [
                boards_row,
                widgets.HBox([move_type_badge], layout=widgets.Layout(justify_content="center")),
                text
            ]
        )
    )



In [20]:
position_features_prompt = """
You are a chess engine. 

Describe the position in terms of these aspects: 
1. Central control
2. Piece activity
3. King safety
4. Pawn structure
5. Initiative

Use 1 sentence per aspect.

Position:
{analysis}
"""

def llm_position_analysis(stockfish_text: str,llm_func) -> str:
    prompt = position_features_prompt.format(
        analysis=stockfish_text
    )
    return llm_func(prompt)

In [21]:
final_analyze_prompt = """
You are a chess position evaluator.

Win probablilities are from White’s perspective.

Compare the following positions (before and after the move):
- Decide if the move is good or bad.
- If good, explain why it improves the position.
- If bad, explain why it weakens the position.
- Use 2-3 sentences in plain language.
- Do NOT mention moves, numbers, or evaluations.

Before:
{before_position}

After:
{after_position}

{sample}
"""
def llm_move_analysis(before_llm, after_llm, sample,llm_func):
    prompt = final_analyze_prompt.format(
        before_position=before_llm,
        after_position=after_llm,
        sample=sample,
    )
    return llm_func(prompt)

In [22]:
def full_llm_pipeline(sample,llm_func, visualize=True):
    # Step 1: LLM evaluates positions independently
    before_llm = llm_position_analysis(sample["before"],llm_func)
    after_llm  = llm_position_analysis(sample["after"],llm_func)

    # Step 2: LLM compares its OWN outputs + Stockfish eval
    final_eval = llm_move_analysis(
        before_llm,
        after_llm,
        sample,
        llm_func
    )
    if visualize:
        show_analysis(sample, final_eval)

    return {
        "before_llm": before_llm,
        "after_llm": after_llm,
        "final_analysis": final_eval
    }
    
def simple_method(sample,llm_func, visualize=True):
    prompt = f"""
Analyze the following move and position.
Rules:
- Write only 2 or 3 sentences.
- Do NOT mention move names.
- Do NOT mention numbers, evaluations, or scores.
- Use plain natural language.

{sample["before"]}
{sample["after"]}

Eval before: {sample['before']["eval"]}
Eval after: {sample['after']["eval"]}
"""
    final_eval = llm_func(prompt)
    if visualize:
        show_analysis(sample, final_eval)
        
    return final_eval


In [23]:
from llama_cpp import Llama

gguf_llm = Llama(
    model_path="/kaggle/working/chess-q8_0.gguf",
    n_ctx=4096,
    n_threads=8,
    temperature=0.35,
    top_p=0.9,
    repeat_penalty=1.15,
    verbose=False
)

def chat_gguf(prompt: str, max_tokens: int = 64) -> str:
    """
    Qwen-style GGUF chat, faithful to llama-cli usage
    """

    full_prompt = (
        "<|im_start|>user\n"
        ">> " + prompt.strip() + "\n"
        ">> <|im_end|>\n"
        ">> <|im_start|>assistant"
    )

    output = gguf_llm(
        full_prompt,
        max_tokens=max_tokens,
        stop=["<|im_end|>"],   # reverse-prompt equivalent
    )

    text = output["choices"][0]["text"].strip()

    # Safety cleanup: stop at first sentence end if needed
    for end in [".", "?", "!"]:
        if end in text:
            text = text[:text.rfind(end) + 1]
            break

    return text


llama_context: n_ctx_per_seq (4096) < n_ctx_train (40960) -- the full capacity of the model will not be utilized


## load the sample data with stockfish labels

In [24]:
samples = [
  {
    "before": {
      "fen": "r2qk2r/ppp2ppp/2n5/3n4/Q1B5/P1P1BP2/5P1P/R3K2R w KQkq - 0 14",
      "stockfish_analysis": "[White POV]: 86.87% cp:3.78 O-O-O Qf6 Rxd5 O-O Qc2 Ne7 Rd3 Ng6 Bb3 c5 Bxc5 Rfc8 Be3 Ne5 (Depth: 21)\n[White POV]: 85.19% cp:3.5 Rd1 Nxc3 Rxd8+ Rxd8 Qc2 Rd1+ Qxd1 Nxd1 Kxd1 Ke7 Bg5+ f6 Re1+ Kd7 Be3 Rd8 Kc1 g6 f4 f5 (Depth: 21)\n[White POV]: 31.53% cp:-1.55 Qb3 Nxe3 Bxf7+ Kf8 fxe3 Qe7 Bd5 Qxe3+ Kf1 Re8 Qc4 g6 Rd1 Re5 Qd3 Qc5 Bxc6 Qxc6 Qd8+ Kf7 (Depth: 21)",
      "eval": "86.87",
      "player_turn": "White"
    },
    "after": {
      "fen": "r2qk2r/ppp2ppp/2n5/3B4/Q7/P1P1BP2/5P1P/R3K2R b KQkq - 0 14",
      "stockfish_analysis": "[White POV]: 20.58% cp:-2.7 Qxd5 Qe4+ Qxe4 fxe4 O-O-O Ke2 Rhe8 Rag1 g6 f3 f5 exf5 gxf5 f4 Na5 Kf2 Rd3 Bd4 Rd2+ Kg3 (Depth: 18)\n[White POV]: 89.52% cp:4.29 O-O Bxc6 Qf6 Be4 Qxc3+ Ke2 f5 Rhc1 Qf6 Bxb7 f4 Bxa8 fxe3 Be4 exf2 Rab1 (Depth: 18)\n[White POV]: 92.2% cp:4.94 Qf6 Bxc6+ bxc6 Qe4+ Qe6 Qxe6+ fxe6 Rg1 O-O O-O-O Rf5 Rd7 g6 Kc2 Ra5 f4 Rxa3 (Depth: 18)",
      "eval": "20.58",
      "player_turn": "Black"
    },
    "move_number": 1,
    "move_type": "blunder"
  },
  {
    "before": {
      "fen": "rnbq1rk1/p1p1bppp/1p2pn2/3p4/2PP1B2/2N1PN2/PP3PPP/R2QKB1R w KQ - 0 7",
      "stockfish_analysis": "[White POV]: 53.36% cp:0.27 Qc2 c5 dxc5 bxc5 Be2 Qb6 O-O Bb7 cxd5 Nxd5 Bg3 Nd7 Rad1 Rfd8 (Depth: 18)\n[White POV]: 53.12% cp:0.25 Rc1 c5 dxc5 bxc5 Be2 Bb7 O-O Nbd7 h3 Qb6 cxd5 Nxd5 Bg3 Nxc3 Rxc3 Nf6 (Depth: 18)\n[White POV]: 52.49% cp:0.2 Bd3 dxc4 Bxc4 Bb7 Qe2 a6 a3 Nbd7 Rd1 b5 Ba2 Qc8 O-O c5 Rc1 c4 e4 Nh5 (Depth: 18)",
      "eval": "53.36",
      "player_turn": "White"
    },
    "after": {
      "fen": "rnbq1rk1/p1p1bppp/1p2pn2/3p4/2PP1B2/2NBPN2/PP3PPP/R2QK2R b KQ - 1 7",
      "stockfish_analysis": "[White POV]: 53.49% cp:0.28 dxc4 Bxc4 Bb7 O-O a6 Qe2 b5 Bd3 Nbd7 Rfd1 Rc8 Ne5 Qe8 a4 b4 Nb1 c5 Nd2 cxd4 exd4 (Depth: 21)\n[White POV]: 53.61% cp:0.29 Ba6 cxd5 Nxd5 Nxd5 Qxd5 O-O Bxd3 Qxd3 Rc8 e4 Qb7 Rad1 Nd7 Rfe1 Re8 h4 Rad8 Qc4 c6 Rd3 (Depth: 21)\n[White POV]: 55.1% cp:0.41 c5 cxd5 Nxd5 Nxd5 exd5 Ne5 Nd7 Nc6 Qe8 Nxe7+ Qxe7 O-O c4 Bc2 Nf6 Qf3 Rd8 (Depth: 21)",
      "eval": "53.49",
      "player_turn": "Black"
    },
    "move_number": 1,
    "move_type": "normal"
  },
  {
    "before": {
      "fen": "rnb1kb1r/pp3ppp/1qp2p2/8/3P4/8/PPPB1PPP/R2QKBNR w KQkq - 0 8",
      "stockfish_analysis": "[White POV]: 56.34% cp:0.51 Nf3 Be6 a3 g6 c4 Bg7 Be2 O-O O-O Nd7 b4 Qd8 Be3 Nb6 Qb3 Re8 Rad1 (Depth: 20)\n[White POV]: 54.36% cp:0.35 c3 Bd6 Bd3 O-O Qc2 h6 Ne2 c5 dxc5 Bxc5 O-O Nc6 Nf4 Ne5 Bh7+ Kh8 b4 Bd6 Be4 Qc7 (Depth: 20)\n[White POV]: 51.24% cp:0.1 Bc3 Bd6 Bd3 Na6 a3 Nc7 Qf3 Nd5 Ne2 Nxc3 bxc3 Qc7 c4 (Depth: 20)",
      "eval": "56.34",
      "player_turn": "White"
    },
    "after": {
      "fen": "rnb1kb1r/pp3ppp/1qp2p2/8/3P4/1P6/P1PB1PPP/R2QKBNR b KQkq - 0 8",
      "stockfish_analysis": "[White POV]: 43.53% cp:-0.52 Qxd4 Nf3 Qe4+ Be2 Bc5 O-O O-O Re1 Rd8 Bc4 Qg6 Bd3 Qh5 h3 Nd7 Bc3 a5 a3 a4 Re4 (Depth: 18)\n[White POV]: 48.75% cp:-0.1 Bd6 c3 O-O Bd3 c5 d5 Nd7 Qc2 Re8+ Ne2 c4 bxc4 g6 O-O Nc5 Nd4 Bd7 Nb5 Bxb5 cxb5 (Depth: 18)\n[White POV]: 51.24% cp:0.1 Be6 Nf3 c5 Be3 cxd4 Qxd4 Bb4+ c3 Qxd4 Bxd4 Ba3 Bb5+ Nc6 Be3 O-O-O (Depth: 18)",
      "eval": "43.53",
      "player_turn": "Black"
    },
    "move_number": 1,
    "move_type": "inaccuracy"
  },
  {
    "before": {
      "fen": "r4r1k/pp1nQppp/1qp2p2/5b2/2BP4/1PP5/P2B1PPP/R3K1NR w KQ - 3 13",
      "stockfish_analysis": "[White POV]: 91.98% cp:4.88 Ne2 Rae8 Qd6 Qa5 Qg3 Rxe2+ Kxe2 Re8+ Kf1 c5 Qf4 Bg6 h4 Kg8 Qf3 (Depth: 21)\n[White POV]: 90.55% cp:4.52 Qe2 Qa5 Qf3 Be4 Qh3 Bf5 Qg3 Qa3 Bc1 Rfe8+ Ne2 Qe7 Qe3 Nb6 Qxe7 Rxe7 Be3 Nxc4 bxc4 Be6 (Depth: 21)\n[White POV]: 89.93% cp:4.38 Qa3 Rfe8+ Ne2 Qd8 O-O Nb6 Rfe1 Nxc4 bxc4 Bd3 Ng3 Rxe1+ Rxe1 Bxc4 Nf5 Qd7 Ne3 Be6 Qb2 b5 (Depth: 21)",
      "eval": "91.98",
      "player_turn": "White"
    },
    "after": {
      "fen": "r4r1k/pp1nQppp/1qp2p2/5b2/2BP4/1PP5/P2B1PPP/2KR2NR b - - 4 13",
      "stockfish_analysis": "[White POV]: 80.45% cp:2.83 c5 dxc5 Qa5 Kb2 Rfe8 b4 Qa4 Bb3 Qxb3+ axb3 Rxe7 Nf3 Ne5 Nxe5 fxe5 Rhe1 Rd7 c4 f6 Bc3 (Depth: 19)\n[White POV]: 83.34% cp:3.22 Rfe8 Qa3 Qc7 Ne2 b5 Bf4 Qc8 Ng3 Nb6 Bf1 Nd5 Bd2 a5 Nxf5 Qxf5 Qb2 Qxf2 c4 b4 Bd3 (Depth: 19)\n[White POV]: 83.95% cp:3.31 Qa5 Kb2 Rfe8 Qa3 Qc7 Nf3 b5 Nh4 bxc4 Nxf5 Reb8 Rhe1 c5 Kc2 cxb3+ axb3 a5 Ra1 cxd4 Nxd4 (Depth: 19)",
      "eval": "80.45",
      "player_turn": "Black"
    },
    "move_number": 1,
    "move_type": "mistake"
  },
  {
    "before": {
      "fen": "r4r1k/2qn1ppp/2p2p2/pp3b2/3P2P1/QPP2N1P/P2B1P2/2KR1B1R b - - 0 18",
      "stockfish_analysis": "[White POV]: 75.85% cp:2.29 Be4 Bg2 b4 Qb2 a4 Rhe1 axb3 axb3 bxc3 Bxc3 Bd5 Nh4 Be6 Nf5 Rfb8 d5 cxd5 Re3 Nb6 Qc2 (Depth: 19)\n[White POV]: 75.95% cp:2.3 b4 Qb2 Be4 Bg2 a4 Rhe1 axb3 axb3 bxc3 Bxc3 Bd5 Nh4 Be6 Nf5 Rfb8 d5 cxd5 Re3 Rc8 Kd2 (Depth: 19)\n[White POV]: 81.3% cp:2.94 Be6 Qb2 a4 b4 Nb6 Kb1 Bd5 Bg2 Nc4 Qc1 c5 Ne1 Bxg2 Nxg2 cxb4 cxb4 Qb7 Ne3 Rfc8 Nxc4 (Depth: 19)",
      "eval": "75.85",
      "player_turn": "Black"
    },
    "after": {
      "fen": "r4r1k/2qn1ppp/2p2p2/pp6/3Pb1P1/QPP2N1P/P2B1P2/2KR1B1R w - - 1 19",
      "stockfish_analysis": "[White POV]: 74.93% cp:2.19 Bg2 b4 Qb2 a4 Rhe1 f5 gxf5 axb3 axb3 bxc3 Qxc3 Bd5 Re3 Qa7 Rde1 f6 (Depth: 19)\n[White POV]: 63.64% cp:1.12 Be2 c5 Rhe1 Qb7 d5 b4 Qb2 a4 Nh4 Ne5 f3 axb3 axb3 Bd3 Bf4 c4 Bxe5 fxe5 bxc4 Bxe2 (Depth: 19)\n[White POV]: 41.7% cp:-0.67 Rg1 Bxf3 Re1 a4 c4 axb3 Qxb3 c5 Qxf3 cxd4 Qe4 Rxa2 Bd3 g6 Qxd4 Ne5 f4 Nxd3+ Qxd3 (Depth: 19)",
      "eval": "74.93",
      "player_turn": "White"
    },
    "move_number": 2,
    "move_type": "normal"
  },
  {
    "before": {
      "fen": "r4r1k/2qn1ppp/2p2p2/p7/1p1Pb1P1/QPP2N1P/P2B1PB1/2KR3R w - - 0 20",
      "stockfish_analysis": "[White POV]: 75.11% cp:2.21 Qb2 a4 Rhe1 axb3 axb3 f5 gxf5 bxc3 Bxc3 Bd5 Bh1 f6 Nd2 Nb6 Bxd5 Nxd5 Nc4 Rfc8 Rd3 c5 (Depth: 20)\n[White POV]: 42.55% cp:-0.6 cxb4 axb4 Qxb4 c5 Qc3 Rxa2 Rde1 Rc2+ Qxc2 Bxc2 Kxc2 cxd4+ Kb2 Qb6 Ra1 Nc5 Ra3 Nd3+ Ka1 Qd6 (Depth: 20)\n[White POV]: 13.94% cp:-3.64 Rhe1 Bxf3 cxb4 Bxg2 bxa5 Bf3 Bb4 Bxd1 Bd6 Qc8 Bxf8 Nxf8 Rxd1 Kg8 Re1 Ne6 Qd6 h6 Kb2 Rxa5 (Depth: 20)",
      "eval": "75.11",
      "player_turn": "White"
    },
    "after": {
      "fen": "r4r1k/2qn1ppp/2p2p2/p7/1P1Pb1P1/QP3N1P/P2B1PB1/2KR3R b - - 0 20",
      "stockfish_analysis": "[White POV]: 38.69% cp:-0.92 axb4 Qxb4 Rxa2 Rde1 c5 Qc3 Rc2+ Qxc2 Bxc2 Kxc2 cxd4+ Kb2 Qb6 Ra1 Nc5 Ra3 Nd3+ Ka1 Qd6 Ra4 (Depth: 18)\n[White POV]: 79.73% cp:2.74 a4 bxa4 c5 dxc5 Nxc5 bxc5 Rab8 Bb4 Qf4+ Rd2 Bxf3 Bxf3 Rxb4 Bd5 Rfb8 Bb3 Qf3 Rhd1 Qc3+ Kb1 (Depth: 18)\n[White POV]: 83.68% cp:3.27 Rfc8 bxa5 c5 Kb2 Qb7 Rhe1 Bxf3 Bxf3 Qxf3 b4 Qb7 dxc5 Nxc5 Ka1 Ne6 Qb2 Qf3 Rc1 Qxh3 Rxc8+ (Depth: 18)",
      "eval": "38.69",
      "player_turn": "Black"
    },
    "move_number": 2,
    "move_type": "blunder"
  },
  {
    "before": {
      "fen": "7k/2q2ppp/5p2/2n5/1Q2R1PN/1P5P/2rB1P2/3K3R b - - 0 27",
      "stockfish_analysis": "[White POV]: 95.77% cp:6.24 Rxd2+ Kxd2 g6 Re3 Qd7+ Ke2 Ne6 Rc1 Kg7 Nf3 h5 Kf1 hxg4 Qxg4 Qd5 Qc4 Qa8 Qe4 Qa6+ Kg1 (Depth: 18)\n[White POV]: 96.73% cp:6.78 Qc8 Re8+ Qxe8 Kxc2 Nd7 Re1 Qc8+ Bc3 h6 Re3 Qa8 Kb2 Ne5 Rxe5 fxe5 (Depth: 18)\n[White POV]: 97.04% cp:6.98 g6 Re8+ Kg7 Kxc2 Qc6 Re3 Ne6+ Qc3 Qxh1 Rxe6 fxe6 f3 Qxh3 Qc7+ Kg8 (Depth: 18)",
      "eval": "95.77",
      "player_turn": "Black"
    },
    "after": {
      "fen": "7k/2q2ppp/5p2/8/1Q2n1PN/1P5P/2rB1P2/3K3R w - - 0 28",
      "stockfish_analysis": "Mate in 1 moves: Qf8# (Depth: 18)\n[White POV]: 96.49% cp:6.63 Qxe4 Rxd2+ Kxd2 g6 Ke2 Kg8 Nf3 h5 Kf1 Qc3 Qe3 Qc8 Qd4 Qc1+ Kg2 hxg4 Rxc1 gxh3+ Kh2 (Depth: 18)\n[White POV]: 87.92% cp:3.97 Nf3 Nxf2+ Ke2 h6 Kxf2 Qa7+ Nd4 Ra2 Re1 Kh7 Re3 Qa8 Nf3 (Depth: 18)",
      "eval": "100.0",
      "player_turn": "White"
    },
    "move_number": 3,
    "move_type": "blunder"
  },
  {
    "before": {
      "fen": "r1b1r1k1/1p1n1ppp/p3pq2/3p4/P1PP4/3BPN2/5PPP/1R1Q1RK1 b - - 0 14",
      "stockfish_analysis": "[White POV]: 54.73% cp:0.38 dxc4 Bxc4 b6 d5 e5 Qb3 Rb8 Qa3 h6 Rfc1 Qg6 Qb4 Bb7 h3 e4 Nd4 Ne5 Qxb6 Qxb6 Rxb6 (Depth: 19)\n[White POV]: 65.7% cp:1.3 Qd8 Qc2 g6 cxd5 exd5 Rfc1 Nf8 Rb4 Re7 Qb3 Re6 Be2 Rd6 h3 Rb8 Ne5 (Depth: 19)\n[White POV]: 66.59% cp:1.38 g6 cxd5 exd5 Qb3 Qd6 Rfc1 Nf6 h3 Re6 Ne5 Ne8 a5 Re7 Qa2 Be6 Rb6 Qd8 Rc5 (Depth: 19)",
      "eval": "54.73",
      "player_turn": "Black"
    },
    "after": {
      "fen": "r1b1r1k1/1p1n1ppp/p4q2/3pp3/P1PP4/3BPN2/5PPP/1R1Q1RK1 w - - 0 15",
      "stockfish_analysis": "[White POV]: 71.8% cp:1.87 dxe5 Nxe5 Nxe5 Rxe5 cxd5 Bf5 Qb3 Bxd3 Qxd3 Rd8 Rfd1 h5 d6 Rd7 Rb4 g6 h3 Rc5 Rd4 Qe5 (Depth: 18)\n[White POV]: 69.52% cp:1.65 Nxe5 Nxe5 dxe5 Rxe5 cxd5 Bf5 Qb3 Bxd3 Qxd3 Rd8 Rfd1 Rexd5 Qxd5 Rxd5 Rxd5 h5 Rd7 b6 Rb7 Qc6 (Depth: 18)\n[White POV]: 56.46% cp:0.52 Be2 dxc4 Bxc4 Rb8 Bd5 b5 h3 h6 axb5 axb5 Qb3 b4 Rfd1 exd4 Nxd4 (Depth: 18)",
      "eval": "71.8",
      "player_turn": "White"
    },
    "move_number": 2,
    "move_type": "mistake"
  },
  {
    "before": {
      "fen": "r1b1r1k1/1p1n1ppp/p4q2/3P4/P2Pp3/3BPN2/5PPP/1R1Q1RK1 w - - 0 16",
      "stockfish_analysis": "[White POV]: 32.4% cp:-1.47 Be2 exf3 Bxf3 b5 e4 Qb6 Qd2 Rb8 axb5 Nf8 h4 axb5 Rb4 Bd7 Rfb1 Qg6 (Depth: 18)\n[White POV]: 30.25% cp:-1.67 Bxe4 Rxe4 Re1 Rb8 Qc2 Re8 e4 b5 axb5 axb5 h3 Qb6 Rb3 Nf8 Ne5 Bd7 Qd3 Qd6 Qf3 f6 (Depth: 18)\n[White POV]: 26.6% cp:-2.03 Re1 exd3 e4 Qd6 Qxd3 f5 Nd2 fxe4 Nxe4 Qg6 d6 Rf8 Qg3 Qxg3 hxg3 b6 Ng5 Nf6 Rxb6 Bd7 (Depth: 18)",
      "eval": "32.4",
      "player_turn": "White"
    },
    "after": {
      "fen": "r1b1r1k1/1p1n1ppp/p4q2/3P4/P2PB3/4PN2/5PPP/1R1Q1RK1 b - - 0 16",
      "stockfish_analysis": "[White POV]: 29.73% cp:-1.72 Rxe4 Re1 Rb8 Qc2 Re8 e4 b5 axb5 axb5 h3 Qd8 Rb3 Nf6 Ne5 Bd7 d6 b4 Qc7 Qxc7 dxc7 (Depth: 18)\n[White POV]: 82.7% cp:3.13 Qd8 Ne5 Nxe5 dxe5 Rxe5 Qd4 Qe8 Bf3 b5 axb5 axb5 Rbc1 Rb8 d6 b4 Rc7 b3 d7 Bxd7 Rxd7 (Depth: 18)\n[White POV]: 85.25% cp:3.51 Qd6 Nd2 Nf6 Bf3 Bf5 Nc4 Qc7 Rc1 Rac8 a5 Qd8 Qd2 (Depth: 18)",
      "eval": "29.73",
      "player_turn": "Black"
    },
    "move_number": 3,
    "move_type": "normal"
  }
]

## testing the model with huggingface format **on gpu**

### simple method

In [25]:
for sample in samples:
    simple_method(sample,chat_qwen3_direct)

### chained method

In [26]:
for sample in samples:
    full_llm_pipeline(sample,chat_qwen3_direct)

## testing the model with gguf format **on cpu**

### simple method

In [27]:
# print(simple_method(samples[6],chat_gguf))

### chained method

In [28]:
# import json

# result = full_llm_pipeline(samples[6], chat_gguf)
# print(json.dumps(result, indent=2, ensure_ascii=False))